# * Ex-D Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250201
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        CASE WHEN REGEXP_LIKE(METRIC_CD, '^DB1R000900|^DB1R001000|^DB1S000101|^DB1S000109|^DB2R000500|^DB2S000100') 
            OR METRIC_CD IN (
                'DB0R00010001' --Total Inflow M1 : DTAC'
                , 'DB1S000100' --Prepay New sub
                , 'DB2S010100' --Postpaid activation sub B2C
                , 'DB2S020100' --Postpaid activation sub B2B
                --, 'DB1S000102' --Prepaid Activation Subs : Thai Mass : DTAC
                --, 'DB1S000103' --Prepaid Activation Subs : AEC/Migrants : DTAC
                --, 'DB1S000104' --Prepaid Activation Subs : Tourists (IN + OUT) : DTAC
                ) THEN 'Obsolete'
            ELSE 'Active'
            END KPI_FLAG
        , PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'A[A-K]$') THEN SUBSTR(METRIC_CD, -2) ELSE 'ALL' END CHANNEL_CD
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) START_DAY, MAX(TM_KEY_DAY) END_DAY
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    GROUP BY PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
    --ORDER BY 1, PAR_KEY, BASENAME, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-05-14, 11:42:35

   -> Summany DataFrame : 123 rows, 20 columns

TDMDBPR : Disconnected


In [3]:
active_df = chk_src_df.loc[chk_src_df['KPI_FLAG']=='Active'].copy()
active_df = active_df.fillna(0).sort_values(by=['KPI_FLAG', 'BASENAME', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = active_df.iloc[:, 9:17].columns.tolist()
for col in mod_col_list:
    active_df[col] = active_df[col].apply(lambda x: format(x, ',.0f'))

# active_df
print(f'\n{active_df.to_string(max_cols=100)}')


   KPI_FLAG   PAR_KEY                              BASENAME    METRIC_CD                                           METRIC_NAME CHANNEL_CD           LOAD_DATE  START_DAY   END_DAY              P              G               H              HH CCAA       H_080 H_016 H_040  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0    Active  20250512                                     0   DB2S000201                  Postpaid Churn Subs Voluntary : DTAC        ALL 2025-05-13 20:28:48   20250503  20250512          1,428          2,856           2,854           2,854    0          29     0     0              4          170     1579
1    Active  20250512                                     0   DB2S000202                Postpaid Churn Subs Involuntary : DTAC        ALL 2025-05-13 20:28:49   20250503  20250512             44             88              88              88    0           1     0     0              4          170     1579
2    Active  20250512  D_CORPKPI_ACTUAL_01_20250512.txt.gpg   DB1R000100      

In [4]:
obsolete_df = chk_src_df.loc[chk_src_df['KPI_FLAG']=='Obsolete'].copy()
obsolete_df = obsolete_df.loc[obsolete_df['CHANNEL_CD']=='ALL']
obsolete_df = obsolete_df.fillna(0).sort_values(by=['KPI_FLAG', 'BASENAME', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = obsolete_df.iloc[:, 9:17].columns.tolist()
for col in mod_col_list:
    obsolete_df[col] = obsolete_df[col].apply(lambda x: format(x, ',.0f'))

# obsolete_df
print(f'\n{obsolete_df.to_string(max_cols=100)}')


   KPI_FLAG   PAR_KEY                              BASENAME     METRIC_CD                      METRIC_NAME CHANNEL_CD  LOAD_DATE  START_DAY   END_DAY           P            G            H           HH CCAA      H_080 H_016 H_040  CNT_AREA_TYPE  CNT_AREA_CD  ROW_CNT
0  Obsolete  20250512  D_CORPKPI_ACTUAL_01_20250512.txt.gpg  DB0R00010001           Total Inflow M1 : DTAC        ALL 2025-05-12   20250503  20250512  46,322,299   92,644,599  185,179,314   92,549,724    0  2,068,606     0     0              4          170     1890
1  Obsolete  20250512  D_CORPKPI_ACTUAL_01_20250512.txt.gpg    DB1R000900         Prepaid Inflow M1 : DTAC        ALL 2025-05-12   20250503  20250512  37,344,945   74,689,891   74,599,808   74,599,808    0    882,790     0     0              4          170     1890
2  Obsolete  20250512  D_CORPKPI_ACTUAL_01_20250512.txt.gpg    DB1R001000         Prepaid Inflow M2 : DTAC        ALL 2025-05-12   20250503  20250512  20,343,197   40,686,394   40,712,314   40,712,314 

In [5]:
''' Revenue '''

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_d_kpi_flag = 'Active'
rev_metric_list = [
    'DB1R000100' #Prepay Revenue
    , 'DB2R000100' #Postpaid Revenue
    , 'DB2R010100' #Postpaid Revenue B2C
    , 'DB2R020100' #Postpaid Revenue B2B
    ]

src_d_rev_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
# src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_d_rev_overall_df = src_d_rev_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_rev_overall_df = src_d_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_rev_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_rev_overall_df[col] = src_d_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_rev_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250512,DB1R000100,Prepay Revenue,2025-05-12,20250503,20250512,"579,512,717","1,159,025,434","1,158,056,799","1,158,056,799",0,"12,561,622",4,170,1850
1,20250512,DB2R000100,Postpaid Revenue,2025-05-12,20250503,20250512,"871,097,686","1,742,195,372","1,740,702,158","1,740,702,158",0,"30,225,462",4,170,1836
2,20250512,DB2R010100,Postpaid Revenue B2C,2025-05-12,20250503,20250512,"848,724,191","1,697,448,383","1,695,960,048","1,695,960,048",0,"30,170,455",4,170,1836
3,20250512,DB2R020100,Postpaid Revenue B2B,2025-05-12,20250503,20250512,"22,373,495","44,746,989","44,742,110","44,742,110",0,"55,007",4,170,1823


In [6]:
''' Subscription '''

src_d_kpi_flag = 'Active'
sub_metric_list = [
    'DB1S000500' #Prepay Usage Subs
    , 'DB1S000600' #Prepay Revenue Subs
    , 'DB2S000500' #Postpaid Active Subs
    , 'DB2S010500' #Postpaid Active Subs B2C
    , 'DB2S020400' #Postpaid Active Sub B2B
    ]

src_d_sub_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
src_d_sub_overall_df = src_d_sub_overall_df.loc[src_d_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_d_sub_overall_df = src_d_sub_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_sub_overall_df = src_d_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_sub_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_sub_overall_df[col] = src_d_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_sub_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250512,DB1S000500,Prepay Usage Subs,2025-05-12,20250503,20250512,"112,813,861","225,627,722","180,384,680","180,384,680",0,"2,004,146",4,170,1850
1,20250512,DB2S000500,Postpaid Active Subs,2025-05-12,20250503,20250512,"57,906,593","115,813,186","92,568,834","92,568,834",0,"1,538,634",4,170,1849
2,20250512,DB2S010500,Postpaid Active Subs B2C,2025-05-12,20250503,20250512,"51,026,652","102,053,304","81,575,019","81,575,019",0,"1,385,405",4,170,1849
3,20250512,DB2S020400,Postpaid Active Sub B2B,2025-05-12,20250503,20250512,"6,879,941","13,759,882","10,993,815","10,993,815",0,"153,229",4,170,1670


## Actual : Transaction
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [7]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    WHERE METRIC_CD IN (
        'DB1R000100' --Prepay Revenue
        , 'DB2R000100' --Postpaid Revenue
        , 'DB2R010100' --Postpaid Revenue B2C
        , 'DB2R020100' --Postpaid Revenue B2B
        , 'DB1S000500' --Prepay Usage Subs
        , 'DB1S000600' --Prepay Revenue Subs
        , 'DB2S000500' --Postpaid Active Subs
        , 'DB2S010500' --Postpaid Active Subs B2C
        , 'DB2S020400' --Postpaid Active Sub B2B
        )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101



TDMDBPR : Connected

   -> Execute query... 2025-05-14, 11:42:37

   -> Daily DataFrame : 80 rows, 16 columns

TDMDBPR : Disconnected


In [8]:
''' DB1R000100 : Prepay Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB1R000100'
print(f'\nDB1R000100 : Prepay Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB1R000100_day__df = chk_src_df.copy()
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['METRIC_CD']==v_metric_cd]
DB1R000100_day__df = DB1R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB1R000100_day__df[col] = DB1R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB1R000100_day__df


DB1R000100 : Prepay Revenue

   -> v_metric_cd: DB1R000100
   -> v_start_day: 20250501


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250512,DB1R000100,Prepay Revenue,20250503,2025-05-12,"62,101,701","124,203,402","124,103,324","124,103,324",0,"1,401,946",0,0,4,170,185
1,20250512,DB1R000100,Prepay Revenue,20250504,2025-05-12,"59,280,894","118,561,788","118,458,386","118,458,386",0,"1,315,014",0,0,4,170,185
2,20250512,DB1R000100,Prepay Revenue,20250505,2025-05-12,"60,823,569","121,647,139","121,540,538","121,540,538",0,"1,314,119",0,0,4,170,185
3,20250512,DB1R000100,Prepay Revenue,20250506,2025-05-12,"60,515,920","121,031,839","120,928,164","120,928,164",0,"1,271,558",0,0,4,170,185
4,20250512,DB1R000100,Prepay Revenue,20250507,2025-05-12,"57,976,077","115,952,155","115,859,319","115,859,319",0,"1,283,653",0,0,4,170,185
5,20250512,DB1R000100,Prepay Revenue,20250508,2025-05-12,"57,770,018","115,540,036","115,443,006","115,443,006",0,"1,288,672",0,0,4,170,185
6,20250512,DB1R000100,Prepay Revenue,20250509,2025-05-12,"55,327,777","110,655,553","110,567,358","110,567,358",0,"1,172,239",0,0,4,170,185
7,20250512,DB1R000100,Prepay Revenue,20250510,2025-05-12,"55,566,535","111,133,070","111,043,981","111,043,981",0,"1,162,885",0,0,4,170,185
8,20250512,DB1R000100,Prepay Revenue,20250511,2025-05-12,"56,736,619","113,473,238","113,373,876","113,373,876",0,"1,237,871",0,0,4,170,185
9,20250512,DB1R000100,Prepay Revenue,20250512,2025-05-12,"53,413,607","106,827,214","106,738,845","106,738,845",0,"1,113,665",0,0,4,170,185


In [9]:
''' DB2R000100 : Postpaid Revenue '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R000100'
print(f'\nDB2R000100 : Postpaid Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R000100_day__df = chk_src_df.copy()
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['METRIC_CD']==v_metric_cd]
DB2R000100_day__df = DB2R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R000100_day__df[col] = DB2R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R000100_day__df


DB2R000100 : Postpaid Revenue

   -> v_metric_cd: DB2R000100
   -> v_start_day: 20250501


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250512,DB2R000100,Postpaid Revenue,20250503,2025-05-12,"90,325,865","180,651,729","180,511,900","180,511,900",0,"3,176,917",0,0,4,170,183
1,20250512,DB2R000100,Postpaid Revenue,20250504,2025-05-12,"88,400,498","176,800,995","176,662,014","176,662,014",0,"3,115,817",0,0,4,170,183
2,20250512,DB2R000100,Postpaid Revenue,20250505,2025-05-12,"85,777,928","171,555,856","171,418,271","171,418,271",0,"3,006,332",0,0,4,170,183
3,20250512,DB2R000100,Postpaid Revenue,20250506,2025-05-12,"89,539,376","179,078,752","178,925,725","178,925,725",0,"3,191,842",0,0,4,170,185
4,20250512,DB2R000100,Postpaid Revenue,20250507,2025-05-12,"86,818,126","173,636,252","173,485,363","173,485,363",0,"2,909,251",0,0,4,170,185
5,20250512,DB2R000100,Postpaid Revenue,20250508,2025-05-12,"86,265,167","172,530,334","172,371,646","172,371,646",0,"3,037,734",0,0,4,170,183
6,20250512,DB2R000100,Postpaid Revenue,20250509,2025-05-12,"88,906,352","177,812,703","177,665,530","177,665,530",0,"2,968,799",0,0,4,170,183
7,20250512,DB2R000100,Postpaid Revenue,20250510,2025-05-12,"89,129,866","178,259,731","178,096,143","178,096,143",0,"3,101,326",0,0,4,170,183
8,20250512,DB2R000100,Postpaid Revenue,20250511,2025-05-12,"83,390,155","166,780,311","166,625,436","166,625,436",0,"2,876,412",0,0,4,170,183
9,20250512,DB2R000100,Postpaid Revenue,20250512,2025-05-12,"82,544,354","165,088,709","164,940,131","164,940,131",0,"2,841,033",0,0,4,170,185


In [10]:
''' DB2R010100 : Postpaid Revenue B2C '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R010100'
print(f'\nDB2R010100 : Postpaid Revenue B2C\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R010100_day__df = chk_src_df.copy()
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['METRIC_CD']==v_metric_cd]
DB2R010100_day__df = DB2R010100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R010100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R010100_day__df[col] = DB2R010100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R010100_day__df


DB2R010100 : Postpaid Revenue B2C

   -> v_metric_cd: DB2R010100
   -> v_start_day: 20250501


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250512,DB2R010100,Postpaid Revenue B2C,20250503,2025-05-12,"87,122,207","174,244,415","174,104,690","174,104,690",0,"3,167,446",0,0,4,170,183
1,20250512,DB2R010100,Postpaid Revenue B2C,20250504,2025-05-12,"87,539,841","175,079,683","174,941,532","174,941,532",0,"3,111,395",0,0,4,170,183
2,20250512,DB2R010100,Postpaid Revenue B2C,20250505,2025-05-12,"84,447,712","168,895,424","168,759,424","168,759,424",0,"3,002,337",0,0,4,170,183
3,20250512,DB2R010100,Postpaid Revenue B2C,20250506,2025-05-12,"87,253,696","174,507,391","174,354,169","174,354,169",0,"3,188,110",0,0,4,170,185
4,20250512,DB2R010100,Postpaid Revenue B2C,20250507,2025-05-12,"84,508,087","169,016,174","168,865,823","168,865,823",0,"2,907,229",0,0,4,170,185
5,20250512,DB2R010100,Postpaid Revenue B2C,20250508,2025-05-12,"84,227,643","168,455,286","168,296,433","168,296,433",0,"3,031,942",0,0,4,170,183
6,20250512,DB2R010100,Postpaid Revenue B2C,20250509,2025-05-12,"84,611,636","169,223,271","169,076,468","169,076,468",0,"2,962,374",0,0,4,170,183
7,20250512,DB2R010100,Postpaid Revenue B2C,20250510,2025-05-12,"86,896,318","173,792,636","173,629,358","173,629,358",0,"3,094,070",0,0,4,170,183
8,20250512,DB2R010100,Postpaid Revenue B2C,20250511,2025-05-12,"81,568,201","163,136,402","162,981,571","162,981,571",0,"2,870,464",0,0,4,170,183
9,20250512,DB2R010100,Postpaid Revenue B2C,20250512,2025-05-12,"80,548,850","161,097,701","160,950,582","160,950,582",0,"2,835,086",0,0,4,170,185


In [11]:
''' DB2R020100 : Postpaid Revenue B2B '''

max_day = chk_src_df['TM_KEY_DAY'].max()
v_start_day = int(str(max_day)[:6]+'01')
# v_start_day = 20240101

v_metric_cd = 'DB2R020100'
print(f'\nDB2R020100 : Postpaid Revenue B2B\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R020100_day__df = chk_src_df.copy()
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['METRIC_CD']==v_metric_cd]
DB2R020100_day__df = DB2R020100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R020100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R020100_day__df[col] = DB2R020100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R020100_day__df


DB2R020100 : Postpaid Revenue B2B

   -> v_metric_cd: DB2R020100
   -> v_start_day: 20250501


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250512,DB2R020100,Postpaid Revenue B2B,20250503,2025-05-12,"3,203,657","6,407,315","6,407,209","6,407,209",0,"9,471",0,0,4,169,182
1,20250512,DB2R020100,Postpaid Revenue B2B,20250504,2025-05-12,"860,656","1,721,313","1,720,482","1,720,482",0,"4,422",0,0,4,167,180
2,20250512,DB2R020100,Postpaid Revenue B2B,20250505,2025-05-12,"1,330,216","2,660,432","2,658,848","2,658,848",0,"3,995",0,0,4,170,183
3,20250512,DB2R020100,Postpaid Revenue B2B,20250506,2025-05-12,"2,285,680","4,571,360","4,571,556","4,571,556",0,"3,731",0,0,4,170,183
4,20250512,DB2R020100,Postpaid Revenue B2B,20250507,2025-05-12,"2,310,039","4,620,077","4,619,541","4,619,541",0,"2,021",0,0,4,170,183
5,20250512,DB2R020100,Postpaid Revenue B2B,20250508,2025-05-12,"2,037,524","4,075,048","4,075,214","4,075,214",0,"5,792",0,0,4,169,182
6,20250512,DB2R020100,Postpaid Revenue B2B,20250509,2025-05-12,"4,294,716","8,589,432","8,589,062","8,589,062",0,"6,425",0,0,4,170,183
7,20250512,DB2R020100,Postpaid Revenue B2B,20250510,2025-05-12,"2,233,547","4,467,095","4,466,785","4,466,785",0,"7,255",0,0,4,170,183
8,20250512,DB2R020100,Postpaid Revenue B2B,20250511,2025-05-12,"1,821,955","3,643,909","3,643,865","3,643,865",0,"5,947",0,0,4,169,182
9,20250512,DB2R020100,Postpaid Revenue B2B,20250512,2025-05-12,"1,995,504","3,991,008","3,989,549","3,989,549",0,"5,947",0,0,4,169,182


In [12]:
# txn_metric_list = [
#     'DB1R000100' #Prepay Revenue
#     , 'DB2R000100' #Postpaid Revenue
#     , 'DB2R010100' #Postpaid Revenue B2C
#     , 'DB2R020100' #Postpaid Revenue B2B
#     , 'DB1S000500' #Prepay Usage Subs
#     , 'DB1S000600' #Prepay Revenue Subs
#     , 'DB2S000500' #Postpaid Active Subs
#     , 'DB2S010500' #Postpaid Active Subs B2C
#     , 'DB2S020400' #Postpaid Active Sub B2B
#     ]